# 第6章　ファクターモデルの導入

In [1]:
using CSV, DataFrames, DataFramesMeta, ShiftedArrays, Statistics, Gadfly, CategoricalArrays, GLM

In [2]:
#ch06_01
monthly_data_df = CSV.File("../../data/ch05_output1.csv", missingstring=["NA"]) |> DataFrame
annual_data_df = CSV.File("../../data/ch05_output2.csv", missingstring=["NA"]) |> DataFrame

,firm_ID,year,R,Re,R_F,industry_ID,sales,OX
,Int64,Int64,Float64?,Float64?,Float64,Int64?,Float64?,Float64?
1,1,2015,missing,missing,0.00743209,missing,missing,missing
2,1,2016,0.997273,0.996708,0.000564989,1,5948.96,564.14
3,1,2017,0.687864,0.687815,4.8848e-5,1,6505.06,691.18
4,1,2018,-0.213613,-0.219399,0.00578611,1,6846.38,751.29
5,1,2019,0.646836,0.647606,-0.000769931,1,7572.24,958.53
6,1,2020,-0.284303,-0.284683,0.000379944,1,7537.63,778.37
7,2,2015,missing,missing,0.00578611,1,3505.75,45.82
8,2,2016,0.375229,0.375998,-0.000769931,1,3491.27,51.25
9,2,2017,0.640732,0.640352,0.000379944,1,3945.66,83.43


In [12]:
#ch06_02
annual_data_df = @chain annual_data_df begin
    @orderby(:firm_ID, :year)
    groupby(:firm_ID)
    @transform(:lagged_ME = lag(:ME))
end

,firm_ID,year,R,Re,R_F,industry_ID,sales,OX
,Int64,Int64,Float64?,Float64?,Float64,Int64?,Float64?,Float64?
1,1,2015,missing,missing,0.00743209,missing,missing,missing
2,1,2016,0.997273,0.996708,0.000564989,1,5948.96,564.14
3,1,2017,0.687864,0.687815,4.8848e-5,1,6505.06,691.18
4,1,2018,-0.213613,-0.219399,0.00578611,1,6846.38,751.29
5,1,2019,0.646836,0.647606,-0.000769931,1,7572.24,958.53
6,1,2020,-0.284303,-0.284683,0.000379944,1,7537.63,778.37
7,2,2015,missing,missing,0.00578611,1,3505.75,45.82
8,2,2016,0.375229,0.375998,-0.000769931,1,3491.27,51.25
9,2,2017,0.640732,0.640352,0.000379944,1,3945.66,83.43


In [13]:
#ch06_03
annual_data_df = @chain annual_data_df begin
    groupby(:year)
    @transform(:w_M = :lagged_ME ./ sum(skipmissing(:lagged_ME)))
end

,firm_ID,year,R,Re,R_F,industry_ID,sales,OX
,Int64,Int64,Float64?,Float64?,Float64,Int64?,Float64?,Float64?
1,1,2015,missing,missing,0.00743209,missing,missing,missing
2,1,2016,0.997273,0.996708,0.000564989,1,5948.96,564.14
3,1,2017,0.687864,0.687815,4.8848e-5,1,6505.06,691.18
4,1,2018,-0.213613,-0.219399,0.00578611,1,6846.38,751.29
5,1,2019,0.646836,0.647606,-0.000769931,1,7572.24,958.53
6,1,2020,-0.284303,-0.284683,0.000379944,1,7537.63,778.37
7,2,2015,missing,missing,0.00578611,1,3505.75,45.82
8,2,2016,0.375229,0.375998,-0.000769931,1,3491.27,51.25
9,2,2017,0.640732,0.640352,0.000379944,1,3945.66,83.43


In [15]:
#ch06_04
annual_data_df = @chain annual_data_df begin
    @rtransform begin
        :w_M = ifelse(:year >= 2016 && ismissing(:w_M), 0, :w_M)
    end
end

,firm_ID,year,R,Re,R_F,industry_ID,sales,OX
,Int64,Int64,Float64?,Float64?,Float64,Int64?,Float64?,Float64?
1,1,2015,missing,missing,0.00743209,missing,missing,missing
2,1,2016,0.997273,0.996708,0.000564989,1,5948.96,564.14
3,1,2017,0.687864,0.687815,4.8848e-5,1,6505.06,691.18
4,1,2018,-0.213613,-0.219399,0.00578611,1,6846.38,751.29
5,1,2019,0.646836,0.647606,-0.000769931,1,7572.24,958.53
6,1,2020,-0.284303,-0.284683,0.000379944,1,7537.63,778.37
7,2,2015,missing,missing,0.00578611,1,3505.75,45.82
8,2,2016,0.375229,0.375998,-0.000769931,1,3491.27,51.25
9,2,2017,0.640732,0.640352,0.000379944,1,3945.66,83.43


In [16]:
#ch06_05
@chain annual_data_df begin
    groupby(:year)
    @combine(:weight_sum = sum(:w_M))
end

,year,weight_sum
,Int64,Float64?
1,2015,missing
2,2016,1.0
3,2017,1.0
4,2018,1.0
5,2019,1.0
6,2020,1.0


In [ ]:
#ch06_06
@chain annual_data_df begin
    @select(:year, :firm_ID, :w_M)
    outerjoin(monthly_data_df, on = [:year, :firm_ID])
    @select()
end

In [ ]:
#ch06_06
factor_data_df = @chain monthly_data_df begin
    @rtransform(:weight_return = :w_M * :R)
    @rsubset(:month_ID >= 13)
    groupby(:month_ID)
    @combine begin
        :R_F = mean(:R_F)
        :R_M = sum(skipmissing(:weight_return))
    end
    @rtransform(:R_Me = :R_M - :R_F)
end

In [ ]:
@chain factor_data_df begin
    plot(x = :R_Me, Geom.histogram)
end

In [ ]:
#ch06_09
@chain factor_data_df begin
    @rtransform(:gross_R_M = 1 + :R_M)
    @transform(:cumulative_gross_R_M = cumprod(:gross_R_M))
    plot(x = :month_ID, y = :cumulative_gross_R_M, Geom.line)
end

In [ ]:
#ch06_10
@chain factor_data_df begin
    @rtransform(:gross_R_M = 1 + :R_M)
    @transform(:cumulative_gross_R_M = cumprod(:gross_R_M))
    @select(:month_ID, :cumulative_gross_R_M)
    push!([12, 1])
    @orderby(:month_ID)
    plot(x = :month_ID, y = :cumulative_gross_R_M, Geom.line)
end

In [ ]:
#ch06_11
annual_data_df = @chain annual_data_df begin
    @rsubset(:year >= 2016)
    groupby(:year)
    @transform(:ME_rank10 = cut(:lagged_ME, 10, labels=1:10))
end

In [ ]:
#ch06_12
@chain annual_data_df begin
    @select(:year, :firm_ID, :ME_rank10)
    dropmissing
    groupby([:year, :ME_rank10])
    @combine(:N = length(:ME_rank10))
end

In [ ]:
#ch06_13
ME_sorted_portfolio_df = @chain annual_data_df begin
    @select(:year, :firm_ID, :ME_rank10)
    outerjoin(monthly_data_df, on = [:year, :firm_ID])
    dropmissing
    groupby([:month_ID, :ME_rank10])
    @combine(:Re = mean(:Re))
end

In [ ]:
@chain ME_sorted_portfolio_df begin
    groupby(:ME_rank10)
    @combine(:mean_Re = mean(:Re))
    plot(x = :ME_rank10, y = :mean_Re, Geom.bar)
end

In [ ]:
#ch06_15
@chain annual_data_df begin
    @rtransform(:lagged_BEME = :lagged_BE / :lagged_ME)
    groupby(:year)
    @transform(:BEME_rank10 = cut(:lagged_BEME, 10, labels=1:10))
    @select(:year, :firm_ID, :BEME_rank10, :lagged_ME)
    outerjoin(monthly_data_df, on = [:year, :firm_ID])
    dropmissing
    groupby([:month_ID, :BEME_rank10])
    @rtransform(:w = :lagged_ME / sum(:lagged_ME))
    groupby([:month_ID, :BEME_rank10])
    @combine begin
        :e_Re = mean(:Re)
        :v_Re = sum.(:w .* :Re)
    end
    groupby(:BEME_rank10)
    @combine begin
        :e_mean_Re = mean(:e_Re)
        :v_mean_Re = mean(:v_Re)
    end
    plot(x = :BEME_rank10, y = :e_mean_Re, Geom.bar)
    #plot(x = :BEME_rank10, y = :v_mean_Re, Geom.bar)
end

In [ ]:
 ME_sorted_portfolio_df = @chain factor_data_df begin
    @select(:month_ID, :R_M, :R_Me)
    outerjoin(ME_sorted_portfolio_df, on = [:month_ID])
end

In [ ]:
mask = ME_sorted_portfolio_df.ME_rank10 .== 1
lm(@formula(Re ~ R_Me), ME_sorted_portfolio_df[mask, :])

In [ ]:
#ch06_19
for i in 1:10
    mask = ME_sorted_portfolio_df.ME_rank10 .== i
    println(lm(@formula(Re ~ R_Me), ME_sorted_portfolio_df[mask, :]))
end